## Project 1 - Tugas CNN - Cars Classification

### Grup 5 - AI 2

Anggota Kelompok: 
* Diah Ayu Setyaningsih
* I Nyoman Warsana
* Iman Santoso

In [1]:
# dataset
# http://ai.stanford.edu/~jkrause/cars/car_dataset.html

# data training
!wget http://imagenet.stanford.edu/internal/car196/cars_train.tgz
# data test
!wget http://imagenet.stanford.edu/internal/car196/cars_test.tgz
# data devkit
!wget http://ai.stanford.edu/~jkrause/cars/car_devkit.tgz

--2019-08-11 22:27:22--  http://imagenet.stanford.edu/internal/car196/cars_train.tgz
Resolving imagenet.stanford.edu (imagenet.stanford.edu)... 171.64.68.16
Connecting to imagenet.stanford.edu (imagenet.stanford.edu)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 979269282 (934M) [application/x-gzip]
Saving to: ‘cars_train.tgz’

cars_train.tgz      100%[===================>] 933.90M  14.6MB/s    in 1m 56s  

2019-08-11 22:30:28 (8.06 MB/s) - ‘cars_train.tgz’ saved [979269282/979269282]

--2019-08-11 22:30:31--  http://imagenet.stanford.edu/internal/car196/cars_test.tgz
Resolving imagenet.stanford.edu (imagenet.stanford.edu)... 171.64.68.16
Connecting to imagenet.stanford.edu (imagenet.stanford.edu)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 977350468 (932M) [application/x-gzip]
Saving to: ‘cars_test.tgz’

cars_test.tgz       100%[===================>] 932.07M  17.2MB/s    in 61s     

2019-08-11 22:31:36 

In [0]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
# Make backup files to drive
# <your sub folder>
# !cp /content/car_devkit.tgz /content/drive/My\ Drive/dtsproject/car_devkit.tgz
# !cp /content/cars_train.tgz /content/drive/My\ Drive/dtsproject/cars_train.tgz
# !cp /content/cars_test.tgz /content/drive/My\ Drive/dtsproject/cars_test.tgz

# Retrieve backup file from drive
!cp /content/drive/My\ Drive/dtsproject/car_devkit.tgz /content/car_devkit.tgz
!cp /content/drive/My\ Drive/dtsproject/cars_train.tgz /content/cars_train.tgz
!cp /content/drive/My\ Drive/dtsproject/cars_test.tgz /content/cars_test.tgz

In [0]:
# Unpack All Data
import tarfile
for file in ['car_devkit.tgz', 'cars_train.tgz', 'cars_test.tgz']:
  tar = tarfile.open(file)
  tar.extractall()
  tar.close

In [3]:
# Create List of Car Names
from scipy.io import loadmat
cars_meta = loadmat('/content/devkit/cars_meta.mat')
cars_list = []
for i in range(len(cars_meta['class_names'][0])):
  cars_list.append(cars_meta['class_names'][0][i][0])
cars_list[:5]

['AM General Hummer SUV 2000',
 'Acura RL Sedan 2012',
 'Acura TL Sedan 2012',
 'Acura TL Type-S 2008',
 'Acura TSX Sedan 2012']

In [0]:
import pandas as pd
import numpy as np

In [5]:
# Create Cars Train Annotations DataFrame
cars_train_info = loadmat('/content/devkit/cars_train_annos.mat')
cars_train_data = [[row.flat[0] for row in line] for line in cars_train_info['annotations'][0]]
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class', 'fname']
cars_train_df = pd.DataFrame(cars_train_data, columns=columns, dtype=str)
cars_train_df.index = np.arange(1, len(cars_train_df)+1)

cars_train_df.to_csv(r'/content/cars_train_annos.csv', index=False)
cars_train_df.head()

,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class,fname
1,39,116,569,375,14,00001.jpg
2,36,116,868,587,3,00002.jpg
3,85,109,601,381,91,00003.jpg
4,621,393,1484,1096,134,00004.jpg
5,14,36,133,99,106,00005.jpg


In [6]:
cars_test_info = loadmat('/content/devkit/cars_test_annos.mat')
cars_test_data = [[row.flat[0] for row in line] for line in cars_test_info['annotations'][0]]
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'fname']
cars_test_df = pd.DataFrame(cars_test_data, columns=columns, dtype=str)
cars_test_df.index = np.arange(1, len(cars_test_df)+1)

cars_test_df.to_csv(r'/content/cars_test_annos.csv', index=False)
cars_test_df.head()

,bbox_x1,bbox_y1,bbox_x2,bbox_y2,fname
1,30,52,246,147,00001.jpg
2,100,19,576,203,00002.jpg
3,51,105,968,659,00003.jpg
4,67,84,581,407,00004.jpg
5,140,151,593,339,00005.jpg


In [7]:
cars_meta_df = pd.DataFrame(cars_list,columns=['Car Name'], dtype=str)
cars_meta_df.index = np.arange(1, len(cars_meta_df)+1)

cars_meta_df.to_csv(r'/content/car_names.csv', index=False)
cars_meta_df.head()

,Car Name
1,AM General Hummer SUV 2000
2,Acura RL Sedan 2012
3,Acura TL Sedan 2012
4,Acura TL Type-S 2008
5,Acura TSX Sedan 2012


In [8]:
from keras.models import Sequential, Model
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

Using TensorFlow backend.


In [9]:
# split to train cars_train data
train_datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.25)

data_train_generator = train_datagen.flow_from_dataframe(
    dataframe= cars_train_df,
    directory= '/content/cars_train/',
    x_col= 'fname',
    y_col= 'class',
    subset= 'training',
    batch_size= 32,
    shuffle= True,
    class_mode= "categorical",
    target_size=(256,256)
)

data_valid_generator = train_datagen.flow_from_dataframe(
    dataframe= cars_train_df,
    directory= '/content/cars_train/',
    x_col= 'fname',
    y_col= 'class',
    subset= 'validation',
    batch_size= 32,
    shuffle= True,
    class_mode= "categorical",
    target_size=(256,256)
)

test_datagen = ImageDataGenerator(rescale=1./255.)

data_test_generator = test_datagen.flow_from_dataframe(
    dataframe= cars_test_df,
    directory= '/content/cars_test/',
    x_col= 'fname',
    y_col= None,
    batch_size= 32,
    shuffle= False,
    class_mode= None,
    target_size=(256,256)
)

Found 6108 validated image filenames belonging to 196 classes.
Found 2036 validated image filenames belonging to 196 classes.
Found 8041 validated image filenames.


In [10]:
# Build CNN Model

# Layer 1
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(256,256,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 2
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

# Layer 3
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Layer 4
model.add(Dense(196))
model.add(Activation('sigmoid'))

# Model Compiling
optimizer= optimizers.Adadelta(lr=0.01)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

W0811 22:33:19.167096 140570794633088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0811 22:33:19.223475 140570794633088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0811 22:33:19.230819 140570794633088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0811 22:33:19.265429 140570794633088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0811 22:33:19.307635 140570794633088 deprecation_wrapp

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 254, 254, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
activation_2 (Activation)    (None, 125, 125, 64)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 41, 41, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 107584)            0         
__________

In [12]:
STEP_SIZE_TRAIN= data_train_generator.n//data_train_generator.batch_size
STEP_SIZE_VALID= data_valid_generator.n//data_valid_generator.batch_size
STEP_SIZE_TEST= data_test_generator.n//data_test_generator.batch_size+1

model.fit_generator(generator=data_train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=data_valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

Epoch 1/10
190/190 [==============================] - 69s 364ms/step - loss: 0.3453 - acc: 0.8477 - val_loss: 0.1099 - val_acc: 0.9947
Epoch 2/10
190/190 [==============================] - 66s 349ms/step - loss: 0.2304 - acc: 0.9074 - val_loss: 0.0504 - val_acc: 0.9949
Epoch 3/10
190/190 [==============================] - 66s 348ms/step - loss: 0.1672 - acc: 0.9382 - val_loss: 0.0382 - val_acc: 0.9949
Epoch 4/10
190/190 [==============================] - 65s 344ms/step - loss: 0.1341 - acc: 0.9540 - val_loss: 0.0358 - val_acc: 0.9949
Epoch 5/10
190/190 [==============================] - 67s 352ms/step - loss: 0.1186 - acc: 0.9616 - val_loss: 0.0355 - val_acc: 0.9949
Epoch 6/10
190/190 [==============================] - 67s 354ms/step - loss: 0.1047 - acc: 0.9684 - val_loss: 0.0358 - val_acc: 0.9949
Epoch 7/10
190/190 [==============================] - 65s 344ms/step - loss: 0.0979 - acc: 0.9717 - val_loss: 0.0360 - val_acc: 0.9949
Epoch 8/10
190/190 [==============================] - 6

In [13]:
model.evaluate_generator(generator=data_valid_generator,
steps=STEP_SIZE_TEST)

[0.03651682950533971, 0.9948979616165161]

In [14]:
data_test_generator.reset()
pred=model.predict_generator(data_test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

252/252 [==============================] - 65s 257ms/step


In [0]:
predicted_class_indices=np.argmax(pred,axis=1)

In [0]:
labels = {i:cars_list[i] for i in range(len(cars_list))}
predictions = [labels[k] for k in predicted_class_indices]

In [0]:
filename=data_test_generator.filenames

In [0]:

results=pd.DataFrame({"Filename":filename,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)

In [19]:
results.head()

,Filename,Predictions
0,00001.jpg,Rolls-Royce Phantom Drophead Coupe Convertible...
1,00002.jpg,Rolls-Royce Phantom Drophead Coupe Convertible...
2,00003.jpg,Rolls-Royce Phantom Drophead Coupe Convertible...
3,00004.jpg,Rolls-Royce Phantom Drophead Coupe Convertible...
4,00005.jpg,Rolls-Royce Phantom Drophead Coupe Convertible...
